In [17]:
import os
import os.path as osp
import json
import pickle

In [27]:
split = 'ScaleQuest-Math'

In [28]:
with open(f'/home/ma-user/workspace/formal_problem_generation/formal_problem_generation/output/autoformalization_pg_kimina7b/{split}.pkl', 'rb') as f:
    data = pickle.load(f)

In [29]:
data_informal = [
    dict(
        informal_problem=d.informal_problem,
        informal_answer=d.informal_answer,
        informal_solution=d.informal_solution,
        header=d.header,
        formal_statement=d.formal_statement,
        metainfo=d.metainfo
    ) for d in data
]

In [30]:
with open(osp.join('/cache/data/fpg_informal_baselines/sampled', f'{split}.jsonl'), 'w') as f:
    for d in data_informal:
        f.write(json.dumps(d)+'\n')